# MI
**_Ahmed - Luisa - Myria_**

## Exercise H6.1 (Convolutional Neural Network)


### 0. Setup and loading MNIST data


In [19]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [16]:
# 1. Importing the MNIST dataset and reshaping it so every image is a 784 vector
(X_train, y_train), (X_holdout, y_holdout) = mnist.load_data()

train_dataset_size = X_train.shape[0]
holdout_dataset_size = X_holdout.shape[0]

image_height = X_train.shape[1]
image_width = X_train.shape[2]

X_train = X_train.reshape(train_dataset_size, (image_height*image_width))
X_holdout = X_holdout.reshape(holdout_dataset_size, (image_height*image_width))

# 2. Change ys to be array of 1s and 0s
y_train = to_categorical(y_train, num_classes=10)
y_holdout = to_categorical(y_holdout, num_classes=10)

X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

((60000, 784), (10000, 784), (60000, 10), (10000, 10))

### 1. Linear model

In [25]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

# 1. Define the model
num_of_nodes = 10
xi      = Input(shape=(image_height*image_width,))   # Input size (784 pixels)
xo      = Dense(num_of_nodes)(xi)                    # Defines a linear model [by default one more dimension is added for bias]
yo      = Activation('softmax')(xo)                  # Defining activation function to be softmax
model   = Model(inputs=[xi], outputs=[yo])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_3 (Dense)             (None, 10)                7850      
                                                                 
 activation_3 (Activation)   (None, 10)                0         
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [28]:
# 2. Define cost/loss function and compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(learning_rate=0.5),  
              metrics=['accuracy'])

# 3. Fitting the model
#     - data: 60,000
#     - batchsize (per one iteration): 100
#     - data contains 600 batches (i.e 600 iterations to cover each datapoint)
#     - 10,000 iterations => 167 epochs
print("ACHTUNG: In what follows:")
print("accuracy: percentage of correctly classified (training) data")
print("val_accuracy: percentage of correctly classified (validation) data")
model.fit(X_train, y_train,
          batch_size=100,
          epochs=167,
          verbose=1,
          validation_data=(X_holdout, y_holdout),
          # validation_split=0.1,
         )

score = model.evaluate(X_holdout, y_holdout, verbose=0)
print('Final validation loss:', score[0])
print('Final validation accuracy:', score[1])

Epoch 1/167
600/600 [==============================] - 1s 2ms/step - loss: 1433.4808 - accuracy: 0.8981 - val_loss: 2060.4275 - val_accuracy: 0.8967
Epoch 2/167
600/600 [==============================] - 1s 1ms/step - loss: 1384.6316 - accuracy: 0.8997 - val_loss: 2073.8347 - val_accuracy: 0.8975
Epoch 3/167
600/600 [==============================] - 1s 1ms/step - loss: 1453.3243 - accuracy: 0.8985 - val_loss: 2354.6958 - val_accuracy: 0.8793
Epoch 4/167
600/600 [==============================] - 1s 1ms/step - loss: 1404.5942 - accuracy: 0.9014 - val_loss: 2124.0735 - val_accuracy: 0.8925
Epoch 5/167
600/600 [==============================] - 1s 1ms/step - loss: 1394.8992 - accuracy: 0.8999 - val_loss: 1939.9434 - val_accuracy: 0.9044
Epoch 6/167
600/600 [==============================] - 1s 1ms/step - loss: 1389.6921 - accuracy: 0.8999 - val_loss: 2364.4607 - val_accuracy: 0.8818
Epoch 7/167
600/600 [==============================] - 1s 1ms/step - loss: 1442.8169 - accuracy: 0.8972 - 